In [1]:
import pandas as pd

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
#먼저 데이터 읽어오기
df=pd.read_excel('../optimal_data2/Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

In [3]:
#스케일링 먼저 하고 ndarray를 dataframe으로 변환
scaler = StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)

Result=[[0 for j in range(4)] for i in range(10)]
Count= int(322/10)*83
for i in range(10):
    #마지막 그룹은 34명
    if i==9:
        X_test=X.iloc[Count*i:-1]
        X_train=X.drop(X.index[Count*i:-1])
        y_test=y.iloc[Count*i:-1]
        y_train=y.drop(y.index[Count*i:-1])
    
    #모든 그룹은 32명씩gi
    X_test=X.iloc[Count*i:Count*(i+1)]
    X_train=X.drop(X.index[Count*i:Count*(i+1)])
    y_test=y.iloc[Count*i:Count*(i+1)]
    y_train=y.drop(y.index[Count*i:Count*(i+1)])
    #모델 정의, 예측
    model = svm.SVC(kernel='rbf') # rbf Kernel
    model.fit(X_train, y_train)
    
    predict=model.predict(X_test)
        
    #Accuracy
    print("[{}]Accuracy : {}".format(i,model.score(X_test,y_test)))

    #f1score
    f1 = f1_score(y_test,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    
    #precision/recall
    list=sk(y_test,predict,average='weighted')
    print("[{}]Precision : {}".format(i,list[0]))
    print("[{}]Recall : {}".format(i,list[1]))
    print("")

    #결과 배열에 넣기
    Result[i][0]=model.score(X_test,y_test)
    Result[i][1]=f1
    Result[i][2]=list[0]
    Result[i][3]=list[1]
    

[0]Accuracy : 0.9145331325301205
[0]F1score : 0.8737073663385103
[0]Precision : 0.8363708504953549
[0]Recall : 0.9145331325301205



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1]Accuracy : 0.9036144578313253
[1]F1score : 0.8578618270550556
[1]Precision : 0.8165190884017999
[1]Recall : 0.9036144578313253



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2]Accuracy : 0.9446536144578314
[2]F1score : 0.9181377963427725
[2]Precision : 0.9319047375944456
[2]Recall : 0.9446536144578314



...

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[8]Accuracy : 0.9314759036144579
[8]F1score : 0.8984293900749197
[8]Precision : 0.8676473590143707
[8]Recall : 0.9314759036144579



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[9]Accuracy : 0.9555722891566265
[9]F1score : 0.9338630996840654
[9]Precision : 0.9131183998040354
[9]Recall : 0.9555722891566265



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0.914533,0.873707,0.836371,0.914533
1,0.903614,0.857862,0.816519,0.903614
2,0.944654,0.918138,0.931905,0.944654
3,0.943524,0.916107,0.890238,0.943524
4,0.852410,0.792503,0.755491,0.852410
5,0.927334,0.892371,0.859949,0.927334
6,0.926581,0.891997,0.880312,0.926581
7,0.894202,0.844257,0.799597,0.894202
8,0.931476,0.898429,0.867647,0.931476
9,0.955572,0.933863,0.913118,0.955572


In [5]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

Average of Accuracy 0.9193900602409638
Average of F1-Score 0.8819234923733366
Average of Precision 0.8551147052149476
Average of Recall 0.9193900602409638
